In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from local_utils import detect_lp
from os.path import splitext,basename
from keras.models import model_from_json
import pytesseract
import glob
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
wpod_net_path = "wpod-net.json"
from keras.preprocessing.image import ImageDataGenerator, load_img

In [2]:
def load_model(path):
    try:
        path = splitext(path)[0]
        with open('%s.json' % path, 'r') as json_file:
            model_json = json_file.read()
        model = model_from_json(model_json, custom_objects={})
        model.load_weights('%s.h5' % path)
        print("Loading model successfully...")
        return model
    except Exception as e:
        print(e)

In [3]:
wpod_net = load_model(wpod_net_path)

Loading model successfully...


In [6]:
import anvil.server

anvil.server.connect("S44CFBK7DAVUJDB6232MJQYF-RZNK4SZD7CYXKASZ")


Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default environment (dev)" as SERVER


In [11]:
import anvil.media
@anvil.server.callable
def imagetotext(file):
    with anvil.media.TempFile(file) as filename:
        img = load_img(filename)
    image = np.array(img)
    img = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    img = img / 255
    img = cv2.resize(img, (224,224))
    Dmax=608
    Dmin=256
    ratio = float(max(img.shape[:2])) / min(img.shape[:2])
    side = int(ratio * Dmin)
    bound_dim = min(side, Dmax)
    _ , LpImg, _, cor = detect_lp(wpod_net, img, bound_dim, lp_threshold=0.5)
    image1 = LpImg[0]
    img_n = cv2.normalize(src=image1, dst=None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    img_n = cv2.cvtColor(img_n, cv2.COLOR_BGR2RGB)
    text = pytesseract.image_to_string(img_n)
    return (text)

Anvil websocket closed (code 1006, reason=Going away)
Reconnecting Anvil Uplink...
Connecting to wss://anvil.works/uplink
Reconnection failed. Waiting 10 seconds, then retrying.
Reconnecting Anvil Uplink...
Connecting to wss://anvil.works/uplink
Reconnection failed. Waiting 10 seconds, then retrying.
Reconnecting Anvil Uplink...
Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default environment (dev)" as SERVER


Exception in thread Thread-172:
Traceback (most recent call last):
  File "C:\Users\Srijan\anaconda3\lib\site-packages\anvil\server.py", line 306, in call
    return _do_call(args, kwargs, fn_name=fn_name)
  File "C:\Users\Srijan\anaconda3\lib\site-packages\anvil\server.py", line 298, in _do_call
    return _threaded_server.do_call(args, kwargs, fn_name=fn_name, live_object=live_object)
  File "C:\Users\Srijan\anaconda3\lib\site-packages\anvil\_threaded_server.py", line 403, in do_call
    raise _server._deserialise_exception(r["error"])
anvil._server.AnvilWrappedError: 'Connection to Anvil Uplink server lost'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Srijan\anaconda3\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "C:\Users\Srijan\anaconda3\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Srijan\anaconda3\lib\site-packages\anvil\serve

Anvil websocket closed (code 1006, reason=Going away)
Reconnecting Anvil Uplink...
Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default environment (dev)" as SERVER
Anvil websocket closed (code 1006, reason=Going away)


Exception in thread Thread-175:
Traceback (most recent call last):
  File "C:\Users\Srijan\anaconda3\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "C:\Users\Srijan\anaconda3\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Srijan\anaconda3\lib\site-packages\anvil\server.py", line 143, in heartbeat_until_reopened
    call("anvil.private.echo", "keep-alive")
  File "C:\Users\Srijan\anaconda3\lib\site-packages\anvil\server.py", line 306, in call
    return _do_call(args, kwargs, fn_name=fn_name)
  File "C:\Users\Srijan\anaconda3\lib\site-packages\anvil\server.py", line 298, in _do_call
    return _threaded_server.do_call(args, kwargs, fn_name=fn_name, live_object=live_object)
  File "C:\Users\Srijan\anaconda3\lib\site-packages\anvil\_threaded_server.py", line 358, in do_call
    send_call()
  File "C:\Users\Srijan\anaconda3\lib\site-packages\anvil\_threaded_server.py", line 352, in send_call
    send_reqresp(req, c

Reconnecting Anvil Uplink...
Connecting to wss://anvil.works/uplink
Reconnection failed. Waiting 10 seconds, then retrying.
Reconnecting Anvil Uplink...
Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default environment (dev)" as SERVER
